想起读大学的时候，被英语四级虐的体无完肤。刚好，疫情期间精进了编程技能，突发奇想，想通过历年四级真题提取词汇，看看都有哪些单词和每个单词的出现频次。说干就干，在网上搜索了一通，没有找到历年真题，后面在某宝上花了4元搞定。因为1989年至2007年共39套真题已合并好，所以时光倒流，就来看看父辈们当时考英语四级的单词情况和现在有什么不同。现在开始我的表演，首先肯定是编程思路，如下：<br><br>
**第一步：**从本地打开真题集文档；<br>
**第二步：**筛选并统计高频词汇；<br>
**第三步：**调用翻译网站进行翻译；<br>
**第四步：**保存数据。

程序如下：

In [ ]:
import requests,json,xlsxwriter,openpyxl,re,collections
from pyquery import PyQuery as pq

def cet4_6(text):
    
    '''
    功能：统计英语四六级高频词汇。
    参数：真题集内容。
    思路：1.用正则表达式在文本中匹配英文单词；
         2.过滤掉单个字母；
         3.把单词全部变为小写。
         4.用collections模块Counter方法统计词频。
    '''

    p = re.compile(r"[a-zA-Z]+")#用正则表达式[a-zA-Z]+匹配所有英文单词
    l = p.findall(text)
    
    p= re.compile(r'\b[a-zA-Z]\b')#匹配所有单个字母
    l1 = p.findall(text)
    
    l2 = []
    for w in l:
        if w in l1:#过滤掉单个字母
            pass
        else:
            l2.append(w.lower())
   

    return collections.Counter(l2).most_common()#用most_common()按降序统计词频并返回


def get_data(word,word_s):
    
    '''
    功能：调用翻译网站进行翻译、提取结果。
    word：单词。
    word_s:频次。
    思路：1.获取源码；
         2.解析源码；
         3.提取数据。
    
    '''
    
    #用requests模块get方法获取网站源码
    url = f'http://dict.youdao.com/w/{word}'
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}
    response = requests.get(url,headers=header)
    
    doc = pq(response.text)#用PyQuery进行解析
    
    #提取数据
    kw_list = []#以列表形式存放翻译结果
    
    keyword = doc('.keyword').text()#单词
    kw_list.append(keyword)

    pronounce = doc('.pronounce').text()#音标
    kw_list.append(pronounce)

    trans_container =doc('.trans-container').eq(0).text()#词义
    trans_container=';'.join(trans_container.split('\n'))
    kw_list.append(trans_container)

    kw_list.append(word_s)#频次
    kw_list.append(word)#原词，防止丢失翻译结果，检测用
    
    print(kw_list)
    save_data(kw_list,path)#调用存储数据函数进行存储数据


def save_data(keyword_list,path):
    
    '''
    功能：存储数据。
    参数：翻译结果。
    path：存储文件路径。
    思路：打开表格--写入数据--保存数据
    '''
    
    #打开表格
    xlsx = openpyxl.load_workbook(path)
    #print(xlsx.get_named_ranges()) # 输出工作页索引范围
    #print(xlsx.get_sheet_names()) # 输出所有工作页的名称
    # 取第一张表
    sheetnames = xlsx.get_sheet_names()
    table = xlsx.get_sheet_by_name(sheetnames[0])
    table = xlsx.active
    #print(table.title) # 输出表名
    nrows = table.max_row # 获得行数
    print(nrows)
    ncolumns = table.max_column # 获得列数
    #print(ncolumns)
    
    #写入数据
    table.cell(nrows+1,1).value = keyword_list[0]#单词
    table.cell(nrows+1,2).value = keyword_list[1]#音标
    table.cell(nrows+1,3).value = keyword_list[2]#词义
    table.cell(nrows+1,4).value = keyword_list[3]#频次
    table.cell(nrows+1,5).value = keyword_list[4]#原词
     
    #保存数据
    xlsx.save(path)
    print('写入成功')
    
    
if __name__ == "__main__":
    
    # 1 创建一个workbook,相当于创建excel文件
    path = '/Users/shining/Documents/GitHub/selfteaching-python-camp/19100302/7Lou/CET4_6.xlsx'
    work_book = xlsxwriter.Workbook(path)
    # 2 创建一个sheet表单
    sheet = work_book.add_worksheet()  
    heads = ['单词', '发音','释义','频次','原词']
    sheet.write_row('A1',heads)
    work_book.close()#建表成功标志
    
    #导入本地真题文档
    with open('/Users/shining/Desktop/大学英语四级(CET-4)历年真题大全[89-07年39套].txt', 'r') as f: 
        text = f.read()
    
    #调用相关函数，执行程序
    l = cet4_6(text)
    for word in l:
        word_k = word[0]
        word_s = word[1]
        get_data(word=word_k,word_s=word_s)
        



['the', '英 [ðə; ði; ðiː] 美 [ðə,ði,ðiː]', 'art. 这；那;adv. 更加（用于比较级，最高级前）', 10844, 'the']
1
写入成功


/Users/shining/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:85: DeprecationWarning: Call to deprecated function get_sheet_names (Use wb.sheetnames).
/Users/shining/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:86: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).


['to', '英 [tə; tu; tuː] 美 [tə,tu,tuː]', 'prep. 朝；位于……；到某处；到某状态；差；给；对于；比；连着；对着；等于；向……表示敬意；伴随；为了给……；适合；按……的看法；（结算账目时）记入;adv. （门等）关上;n. (To) （汤霍特、丹）多（人名）', 5534, 'to']
2
写入成功
['of', '英 [əv; ɒv] 美 [əv,ʌv]', 'prep. 属于；……的；……的一部分；住在（某地）；关于；由……组成的；因为；（表示人或事的时空位置）在，当', 4164, 'of']
3
写入成功
['in', '英 [ɪn; ˈɪn ðət] 美 [ɪn,ˈɪn ðət]', 'prep. 在……内；进……里；在……期间；在……以后；穿着；在……状态中；在……方面；包含在……中；从事……；用……语言，以……媒介；以……调；在（活动）过程中；在……岁数；当……时；（引出具某种品质的人的名字）；占……；在……之中（用于较大和较小数字之间）；以……的数量；带……颜色的；以……方式；具有……特性；按照（表示方式）;adv. 进入；在里头；到达；当选；涨潮；在屋里；在车站；收到；递交，送到；填写，着色；朝里，向中间；击球；（球）落在界内；（服装等）时髦;adj. 在里面的；时髦的；（用球）界内的;n. 执政者；门路;[ 过去式 inned 过去分词 inned 现在分词 inning ]', 3238, 'in']
4
写入成功
['and', '英 [ənd; ən; n; ænd] 美 [ənd,ən,n,ænd]', 'conj. 和，与；就；而且；但是；然后;n. (And)人名；(土、瑞典)安德', 2792, 'and']
5
写入成功
['is', '英 [ɪz; s; z; əz] 美 [ɪz; s; z; əz]', 'v. 是（be的三单形式）;n. 存在', 2258, 'is']
6
写入成功
['are', '英 [ɑ:(r)] 美 [ɑr]', 'v. 是（be的第二人称单复数现在式）;n. 公亩;n. (Are)人名；(意、西、芬)阿雷', 1681, 'are']
7
写入成功
['that', '英 [ðæt] 美 [ðæt]', 'det. 那，那个；那种;pron. 那， 那个；